In [52]:
import numpy as np
import pandas as pd
import nltk
from konlpy.tag import Twitter, Komoran
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

from collections import Counter


In [2]:
with open('movie_reviews_mini.txt', 'rt', encoding='utf-8') as f:
    data = f.readlines()
print(data[:10], len(data))

['아련한 향수를 떠올리게 만드는 추억의 영화\n', 'SF 코미디 영화사에서 가장 주목해야할 영화\n', 'mv00036133의 꿈이 고스란히 담겨진 최고의 영화\n', '전편을 120% 활용하는 천재적인 속편.\n', '1편만큼 재미있다. 3편만 조금 떨어짐.\n', '역시 전편을 능가하는 속편은 지극히 드물다\n', '무슨 설명이 더 필요한가?\n', 'mv00501003라는 이름만으로도 설레게 만드는 영화의 힘\n', '이때참 ac00000559가 멋져 보였지\n', '시원하면서도 심장을 꽉 조여온다.\n'] 10000


In [112]:
twi = Twitter()

#조사, 어미, 알파벳, 숫자, 구두점, 접사
tagger_list_tw = ['Josa', 'Eomi', 'Alpha', 'Number', 'Punctuation', 'Suffix']

def morpheme(data: list):
    #input : 데이터 전체 리스트 
    #return : 형태소 분석된 데이터 리스트(리스트별 한문장), 형태소 분석된 데이터 전체 리스트
    
    senten = []
    into_sent = []
    max_len_sentence = 0
    docs = []
    docs_for_vec = []
    
    for da in data:# 한문장       
        poses = twi.pos(da, norm=True, stem=True) #normalization & stemming
        #print(poses)
        
        for i in range(len(poses)):
            if not i ==len(poses)-1:
                if not poses[i][1] in tagger_list_tw and len(poses[i][0]) != 1: #특정 형태소 제거 & 한글자 제거
                    senten.append(poses[i][0]+'/'+ poses[i][1])
            else:
                if len(senten) > max_len_sentence: #최대길이 문장 구하기
                    max_len_sentence = len(senten)
                into_sent = list(senten)
                senten.clear()
        
        docs.append(into_sent)
        docs_for_vec.extend(into_sent)
#     print(docs_for_vec)
#     print('\n')
#     print(docs)
    print('max length of sentence : ', max_len_sentence)
    return to_vectorize(docs, docs_for_vec)


def to_vectorize(data: list, data_for_vec: list):
    # data : 형태소 분석된 데이터 리스트(리스트별 한문장)
    # data_for_vec : 형태소 분석된 데이터 전체 리스트
    # output : 벡터화된 데이터 전체 리스트
    vec_size = len(data_for_vec)
    print('total size of vectors : ', vec_size)
    print('count vectors : ', len(set(data_for_vec))) 
    
    Word2Vec_model = Word2Vec(data, size=100, window = 2, min_count=1, iter=10, sg=1)
    word_vectors = Word2Vec_model.wv #모델 저장 with KeyedVectors
    del Word2Vec_model
    
    word_vec_li = []
    word_vec_li_LSTM = []
    sentence_vec_li = []
    sentence_vec_li_LSTM = []
    
    for sentence in data:
        for word in sentence:
            word_vec_li.extend(word_vectors[word])
            word_vec_li_LSTM.append(word_vectors[word])
            
        word_vec = list(word_vec_li)
        word_vec_LSTM = list(word_vec_li_LSTM)
        print('MLP char count : ',len(word_vec))
        sentence_vec_li.append(word_vec) #벡터화된 문장별 리스트
        sentence_vec_li_LSTM.append(word_vec_LSTM) #벡터화된 문장별 리스트 for LSTM
        word_vec_li.clear()
        word_vec_li_LSTM.clear()
        
    print(len(sentence_vec_li))
    #print(word_vectors.vocab)
    return sentence_vec_li
    

# def preprocess(data: list, max_length: int):
#     vectorized_data = morpheme(data)
#     zero_padding = np.zeros((len(data), max_length), dtype=np.int32)
#     for idx, seq in enumerate(vectorized_data):
#     length = len(seq)
#     if length >= max_length:
#         length = max_length
#         zero_padding[idx, :length] = np.array(seq)[:length]
#     else:
#         zero_padding[idx, :length] = np.array(seq)
#     return zero_padding
    

    
#     cnt = Counter(data_for_vec)
#     print(cnt.most_common(20))

morpheme(data[:10])


max length of sentence :  6
total size of vectors :  40
count vectors :  36
MLP char count :  500
MLP char count :  500
MLP char count :  300
MLP char count :  500
MLP char count :  300
MLP char count :  600
MLP char count :  300
MLP char count :  400
MLP char count :  300
MLP char count :  300
10


[[0.003728545,
  -0.0007807035,
  0.0013772891,
  0.0044116718,
  0.0035040721,
  0.0021780715,
  -0.00025314136,
  0.0026856014,
  -0.0046094535,
  -0.0022367837,
  -0.0037614424,
  -0.0015957101,
  0.0037110855,
  -0.0049841432,
  -0.0013675817,
  -0.0023163764,
  -0.00013727801,
  0.0020502247,
  0.0022811,
  0.0017949467,
  -0.0014111839,
  -0.0047665681,
  -0.0010391042,
  0.0043003904,
  -0.0013494807,
  0.00078980229,
  -0.0043292097,
  0.0021025841,
  0.00075822294,
  -0.0035411932,
  0.0035579591,
  0.0010627414,
  0.0046470943,
  -0.0042579286,
  -0.0025459067,
  0.0047417572,
  -0.0046071312,
  -0.0044843126,
  -0.0033271259,
  0.0030323251,
  -0.0014388663,
  -0.0022992133,
  0.00010966614,
  -0.00033377318,
  -0.0036489384,
  -0.0047224457,
  -0.0017330702,
  0.0033612978,
  -0.00023172393,
  -0.0034127722,
  -0.0042760153,
  -0.0031950837,
  0.0049072863,
  -0.0047534937,
  -0.00075169047,
  -0.0036441768,
  0.0031560413,
  0.0026650988,
  -0.0012020587,
  -0.00040493277,

In [ ]:
If you're finished training a model (i.e. no more updates, only querying),
then switch to the :mod:`gensim.models.KeyedVectors` instance in wv

  >>> word_vectors = model.wv
  >>> del model

In [74]:
#구문분석
# POS tag a sentence
sentence = u'만 6세 이하의 초등학교 취학 전 자녀를 양육하기 위해서는'
words = Twitter().pos(sentence)

# Define a chunk grammar, or chunking rules, then chunk
grammar = """
NP: {<N.*>*<Suffix>?}   # Noun phrase
VP: {<V.*>*}            # Verb phrase
AP: {<A.*>*}            # Adjective phrase
"""
parser = nltk.RegexpParser(grammar)
chunks = parser.parse(words)
print("# Print whole tree")
print(chunks)

print("\n# Print noun phrases only")
for subtree in chunks.subtrees():
    if subtree.label()=='NP':
        print(' '.join((e[0] for e in list(subtree))))
        print(subtree)

# Display the chunk tree
#chunks.draw()

# Print whole tree
(S
  (NP 만/Noun 6/Number 세/Noun 이하/Noun)
  의/Josa
  (NP 초등학교/Noun 취학/Noun 전/Noun 자녀/Noun)
  를/Josa
  (NP 양육/Noun)
  (VP 하기/Verb 위해서/Verb)
  는/Eomi)

# Print noun phrases only
만 6 세 이하
(NP 만/Noun 6/Number 세/Noun 이하/Noun)
초등학교 취학 전 자녀
(NP 초등학교/Noun 취학/Noun 전/Noun 자녀/Noun)
양육
(NP 양육/Noun)


In [3]:
with open('movie_reviews_mini.txt', 'rt', encoding='utf-8') as f:
    self.reviews = preprocess(f.readlines(), max_length)
    
def preprocess(data: list, max_length: int):
    """
     입력을 받아서 딥러닝 모델이 학습 가능한 포맷으로 변경하는 함수입니다.
     기본 제공 알고리즘은 char2vec이며, 기본 모델이 MLP이기 때문에, 입력 값의 크기를 모두 고정한 벡터를 리턴합니다.
     문자열의 길이가 고정값보다 길면 긴 부분을 제거하고, 짧으면 0으로 채웁니다.

    :param data: 문자열 리스트 ([문자열1, 문자열2, ...])
    :param max_length: 문자열의 최대 길이
    :return: 벡터 리스트 ([[0, 1, 5, 6], [5, 4, 10, 200], ...]) max_length가 4일 때
    """
    vectorized_data = [decompose_str_as_one_hot(datum, warning=False) for datum in data]
    zero_padding = np.zeros((len(data), max_length), dtype=np.int32)
    for idx, seq in enumerate(vectorized_data):
        length = len(seq)
        if length >= max_length:
            length = max_length
            zero_padding[idx, :length] = np.array(seq)[:length]
        else:
            zero_padding[idx, :length] = np.array(seq)
    return zero_padding

RuntimeError: from_numpy expects an np.ndarray but got list

In [ ]:
#빈도 분석
cnt = Counter(doc)
print(cnt.most_common(20))#문장 전체에서하자